In [1]:
import torchvision
import torch
from model.SD import SD

ModuleNotFoundError: No module named 'models'

ModuleNotFoundError: No module named 't'

In [3]:
complete_swin = torchvision.models.swin_t(weights="DEFAULT")
model = torch.nn.Sequential(*list(complete_swin.children())[:-5])

In [4]:
from PIL import Image

transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((256, 256)),  # resize to 224*224
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
        ),  # normalization
    ]
)
img = Image.open(r'IMG_7469.JPG').convert('RGB')
x = transform(img).unsqueeze_(0)
feat = model(x)

In [6]:
from nystrom_attention import NystromAttention

class Nystrom_MSA(torch.nn.Module):
    def __init__(self, norm_layer=torch.nn.LayerNorm, dim=512) -> None:
        super().__init__()
        self.norm = norm_layer(dim)
        self.attn = NystromAttention(
            dim=dim,
            dim_head=dim // 8,
            heads=8,
            num_landmarks=dim // 2,
            pinv_iterations=6,
            residual=True,
            dropout=0.1,
        )
    def forward(self, x):
        out, attn = self.attn(self.norm(x), return_attn=True)
        x = x + out
        return x, out, attn

nystorm_msa = Nystrom_MSA(norm_layer=torch.nn.LayerNorm, dim=768)
x,out,attn = nystorm_msa(ff)

In [7]:
print(x.shape, out.shape, attn.shape)

torch.Size([1, 64, 768]) torch.Size([1, 64, 768]) torch.Size([1, 8, 384, 384])


In [15]:
import math
class PositionalEncoding(torch.nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

a = PositionalEncoding(768,0.1)(x)
print(a.shape)

torch.Size([1, 64, 768])
